In [1]:
import torch
import time
import pandas as pd


# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

Используемое устройство: cpu


In [2]:
#3.1 Подготовка данных
# Создайте большие матрицы размеров:
# - 64 x 1024 x 1024
# - 128 x 512 x 512
# - 256 x 256 x 256
# Заполните их случайными числами
# Создание матриц
matrices = {
    "64x1024x1024": torch.rand(64, 1024, 1024, device="cpu"),
    "128x512x512": torch.rand(128, 512, 512, device="cpu"),
    "256x256x256": torch.rand(256, 256, 256, device="cpu")
}

# Копии матриц на GPU, если доступен
matrices_gpu = {key: mat.to(device) for key, mat in matrices.items()} if torch.cuda.is_available() else {}

In [3]:
#3.2 Функция измерения времени
# Создайте функцию для измерения времени выполнения операций
# Используйте torch.cuda.Event() для точного измерения на GPU
# Используйте time.time() для измерения на CPU
def measure_time_cpu(operation, *args):
    """ 
    Время выполнения на cpu
    """
    start = time.time()
    result = operation(*args)
    end = time.time()
    return (end - start) * 1000  # Время в миллисекундах

def measure_time_gpu(operation, *args):
    """ 
    Время выполнения на gpu
    """
    if not torch.cuda.is_available():
        return float("inf")  # Если GPU недоступен, возвращаем бесконечность
    start_event = torch.cuda.Event(enable_timing=True)
    end_event = torch.cuda.Event(enable_timing=True)
    torch.cuda.synchronize()
    start_event.record()
    result = operation(*args)
    end_event.record()
    torch.cuda.synchronize()
    return start_event.elapsed_time(end_event)  # Время в миллисекундах

In [4]:
#3.3 Сравнение операций
# Сравните время выполнения следующих операций на CPU и CUDA:
# - Матричное умножение (torch.matmul)
# - Поэлементное сложение
# - Поэлементное умножение
# - Транспонирование
# - Вычисление суммы всех элементов

# Для каждой операции:
# 1. Измерьте время на CPU
# 2. Измерьте время на GPU (если доступен)
# 3. Вычислите ускорение (speedup)
# 4. Выведите результаты в табличном виде
# Операции для тестирования
operations = {
    "Матричное умножение": lambda x: torch.matmul(x, x),
    "Поэлементное сложение": lambda x: x + x,
    "Поэлементное умножение": lambda x: x * x,
    "Транспонирование": lambda x: x.transpose(0, 1),
    "Сумма всех элементов": lambda x: x.sum()
}

# Сбор результатов
results = []

for mat_name, mat_cpu in matrices.items():
    mat_gpu = matrices_gpu.get(mat_name, None)
    for op_name, op in operations.items():
        # Измерение времени на CPU
        time_cpu = measure_time_cpu(op, mat_cpu)
        
        # Измерение времени на GPU
        time_gpu = measure_time_gpu(op, mat_gpu) if mat_gpu is not None else float("inf")
        
        # Вычисление ускорения
        speedup = time_cpu / time_gpu if time_gpu != 0 and time_gpu != float("inf") else "N/A"
        
        results.append({
            "Матрица": mat_name,
            "Операция": op_name,
            "CPU (мс)": round(time_cpu, 3),
            "GPU (мс)": round(time_gpu, 3) if time_gpu != float("inf") else "N/A",
            "Ускорение": f"{round(speedup, 2)}x" if isinstance(speedup, float) else speedup
        })

# Вывод результатов в таблице
df = pd.DataFrame(results)
print("\nРезультаты сравнения операций:")
print(df.to_string(index=False))


Результаты сравнения операций:
     Матрица               Операция  CPU (мс) GPU (мс) Ускорение
64x1024x1024    Матричное умножение   608.744      N/A       N/A
64x1024x1024  Поэлементное сложение    39.097      N/A       N/A
64x1024x1024 Поэлементное умножение    38.000      N/A       N/A
64x1024x1024       Транспонирование     1.979      N/A       N/A
64x1024x1024   Сумма всех элементов     9.219      N/A       N/A
 128x512x512    Матричное умножение   121.682      N/A       N/A
 128x512x512  Поэлементное сложение    22.281      N/A       N/A
 128x512x512 Поэлементное умножение    18.720      N/A       N/A
 128x512x512       Транспонирование     0.030      N/A       N/A
 128x512x512   Сумма всех элементов     4.791      N/A       N/A
 256x256x256    Матричное умножение    27.262      N/A       N/A
 256x256x256  Поэлементное сложение    10.053      N/A       N/A
 256x256x256 Поэлементное умножение     9.450      N/A       N/A
 256x256x256       Транспонирование     0.044      N/A    

In [ ]:
#3.4 Анализ результата
"""
 Проанализируйте результаты:
 - Какие операции получают наибольшее ускорение на GPU?
Матричное умножение: Эта операция обычно получает наибольшее ускорение на GPU, так как она вычислительно интенсивна и хорошо параллелизуется.
Поэлементное сложение и умножение: Эти операции также получают значительное ускорение, так как они легко распараллеливаются на GPU.
Транспонирование: Ускорение умеренное, так как транспонирование требует интенсивного доступа к памяти, что может быть узким местом на GPU.
Сумма всех элементов: Ускорение зависит от размера матрицы. Для больших матриц оно может быть значительным, но операция редукции менее эффективна на GPU, чем матричные операции.
 - Почему некоторые операции могут быть медленнее на GPU?
 Низкая вычислительная интенсивность: Операции, такие как транспонирование или сумма, требуют меньше вычислений и больше операций с памятью, что снижает эффективность GPU, так как они оптимизированы для вычислительно интенсивных задач.
 Передача данных: Копирование данных между CPU и GPU может занимать значительное время, особенно для небольших матриц. Если операция сама по себе быстрая (например, транспонирование), накладные расходы на передачу данных могут превысить выгоду от выполнения на GPU.
 - Как размер матриц влияет на ускорение?
 Большие матрицы (например, 64x1024x1024): Ускорение на GPU обычно выше, так как большие объемы данных позволяют лучше использовать параллелизм GPU. Вычислительные операции (например, матричное умножение) выигрывают больше, чем операции с памятью.
 Меньшие матрицы (например, 256x256x256): Ускорение может быть ниже из-за меньшего объема данных, что не позволяет полностью загрузить все ядра GPU. Накладные расходы на передачу данных и запуск ядер становятся более заметными.
 - Что происходит при передаче данных между CPU и GPU?
 Копирование данных: Передача данных между CPU и GPU (например, tensor.to(device)) требует времени, так как данные копируются через шину PCIe. Это может составлять значительную часть общего времени, особенно для небольших матриц или простых операций.
 """